In [1]:
import pandas as pd
import numpy as np

from sklearn.model_selection import(
    cross_validate,
    train_test_split
)

from sklearn.compose import ColumnTransformer, make_column_transformer
from sklearn.dummy import DummyClassifier
from sklearn.impute import SimpleImputer
from sklearn.model_selection import cross_val_score, cross_validate, train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder, StandardScaler
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier

# 1. Reading the dataset

In [2]:
census_df = pd.read_csv('adult.csv')
census_df

,age,workclass,fnlwgt,education,education.num,marital.status,occupation,relationship,race,sex,capital.gain,capital.loss,hours.per.week,native.country,income
0,90,?,77053,HS-grad,9,Widowed,?,Not-in-family,White,Female,0,4356,40,United-States,<=50K
1,82,Private,132870,HS-grad,9,Widowed,Exec-managerial,Not-in-family,White,Female,0,4356,18,United-States,<=50K
2,66,?,186061,Some-college,10,Widowed,?,Unmarried,Black,Female,0,4356,40,United-States,<=50K
3,54,Private,140359,7th-8th,4,Divorced,Machine-op-inspct,Unmarried,White,Female,0,3900,40,United-States,<=50K
4,41,Private,264663,Some-college,10,Separated,Prof-specialty,Own-child,White,Female,0,3900,40,United-States,<=50K
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32556,22,Private,310152,Some-college,10,Never-married,Protective-serv,Not-in-family,White,Male,0,0,40,United-States,<=50K
32557,27,Private,257302,Assoc-acdm,12,Married-civ-spouse,Tech-support,Wife,White,Female,0,0,38,United-States,<=50K
32558,40,Private,154374,HS-grad,9,Married-civ-spouse,Machine-op-inspct,Husband,White,Male,0,0,40,United-States,>50K
32559,58,Private,151910,HS-grad,9,Widowed,Adm-clerical,Unmarried,White,Female,0,0,40,United-States,<=50K


# 2. Data splitting

In [3]:
train_df, test_df = train_test_split(census_df,
                                     train_size=0.8,
                                     random_state=2018)

# 3. Preliminary EDA

We are basically checking for missing values, scaling issues in the dataframe.

In [4]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 26048 entries, 12182 to 9466
Data columns (total 15 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   age             26048 non-null  int64 
 1   workclass       26048 non-null  object
 2   fnlwgt          26048 non-null  int64 
 3   education       26048 non-null  object
 4   education.num   26048 non-null  int64 
 5   marital.status  26048 non-null  object
 6   occupation      26048 non-null  object
 7   relationship    26048 non-null  object
 8   race            26048 non-null  object
 9   sex             26048 non-null  object
 10  capital.gain    26048 non-null  int64 
 11  capital.loss    26048 non-null  int64 
 12  hours.per.week  26048 non-null  int64 
 13  native.country  26048 non-null  object
 14  income          26048 non-null  object
dtypes: int64(6), object(9)
memory usage: 3.2+ MB


Luckily, we do not have NANs values in the dataset. We still have to have to look whether some of the text columns have missing or empty strings in it.

In [5]:
train_df.sort_index()

,age,workclass,fnlwgt,education,education.num,marital.status,occupation,relationship,race,sex,capital.gain,capital.loss,hours.per.week,native.country,income
1,82,Private,132870,HS-grad,9,Widowed,Exec-managerial,Not-in-family,White,Female,0,4356,18,United-States,<=50K
2,66,?,186061,Some-college,10,Widowed,?,Unmarried,Black,Female,0,4356,40,United-States,<=50K
3,54,Private,140359,7th-8th,4,Divorced,Machine-op-inspct,Unmarried,White,Female,0,3900,40,United-States,<=50K
4,41,Private,264663,Some-college,10,Separated,Prof-specialty,Own-child,White,Female,0,3900,40,United-States,<=50K
5,34,Private,216864,HS-grad,9,Divorced,Other-service,Unmarried,White,Female,0,3770,45,United-States,<=50K
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32555,53,Private,321865,Masters,14,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,40,United-States,>50K
32557,27,Private,257302,Assoc-acdm,12,Married-civ-spouse,Tech-support,Wife,White,Female,0,0,38,United-States,<=50K
32558,40,Private,154374,HS-grad,9,Married-civ-spouse,Machine-op-inspct,Husband,White,Male,0,0,40,United-States,>50K
32559,58,Private,151910,HS-grad,9,Widowed,Adm-clerical,Unmarried,White,Female,0,0,40,United-States,<=50K



Usually .info() method would give you information on missing values. But here, it does not pick "?" as missing values as they are encoded as strings instead of an actual NaN in Python. So let's replace them with np.nan before we carry out EDA.

In [6]:
train_df_nan = train_df.replace("?", np.nan)
test_df_nan = test_df.replace("?", np.nan)

In [7]:
train_df.describe()

,age,fnlwgt,education.num,capital.gain,capital.loss,hours.per.week
count,26048.000000,2.604800e+04,26048.000000,26048.000000,26048.000000,26048.000000
mean,38.597743,1.898041e+05,10.072827,1058.883715,87.270769,40.421798
std,13.612159,1.052288e+05,2.574470,7230.138888,402.341325,12.427114
min,17.000000,1.228500e+04,1.000000,0.000000,0.000000,1.000000
25%,28.000000,1.177630e+05,9.000000,0.000000,0.000000,40.000000
50%,37.000000,1.786190e+05,10.000000,0.000000,0.000000,40.000000
75%,48.000000,2.373260e+05,12.000000,0.000000,0.000000,45.000000
max,90.000000,1.484705e+06,16.000000,99999.000000,4356.000000,99.000000


- A lot of the values in `captial.gain` and `capital.loss` are 0's. We can see that even 75% percentile of the value is 0.
- `education.num` is an ordinal feature and the feature `education` becomes redundant.

# 4. EDA

## 4.1 Relationships among the features.

In [8]:
corr_df = train_df.corr('spearman').stack().reset_index(name='corr')
corr_df.loc[corr_df['corr'] == 1, 'corr'] = 0  # Remove diagonal
# Use abs so that we can visualize the impact of negative correaltion  
corr_df['abs'] = corr_df['corr'].abs()
corr_df.sort_values('abs', ascending=False).head(n=5)

,level_0,level_1,corr,abs
32,hours.per.week,education.num,0.166687,0.166687
17,education.num,hours.per.week,0.166687,0.166687
30,hours.per.week,age,0.141171,0.141171
5,age,hours.per.week,0.141171,0.141171
18,capital.gain,age,0.128159,0.128159


In [9]:
alt.Chart(corr_df).mark_circle().encode(
    x='level_0',
    y='level_1',
    size='abs',
    color=alt.Color('corr',
                    scale=alt.Scale(scheme='blueorange',
                                    domain=(-1, 1)))).properties(
    height=150,
    width=150)

alt.Chart(...)

- There are very low correlation between the features. 
- But the above relations are only for numeric features.

# 5. Transformations to be applied on features

- The imputation are done on the categorical columns which have NaN values in them. 
- The numeric columns don't have NaN values. We scale the numeric columns. 
- We are doing OHE on categorical variables. For features having both imputation and OHE, first we impute the feature and then apply OHE.

| Transformation | Features |
| --- | ------ |
| Scaling | age, fnlwgt, capital.gain, capital.loss |
| Imputation | occupation, workclass, native.country |
| OHE | occupation, workclass, education.num, marital.status, relationship, race, sex, native.country |
| drop | education |


# 6. Preprocessing using sklearn's ColumnTransformer

Below we prepar the list of common features for each of the transformations we want to make. For ethical reasons, we would like to drop `race` and `sex` from our analysis. For some scenarios, it might be fine to include them for descriptive purposes.

In [10]:
numeric_features = ['age', 'fnlwgt', 'capital.gain', 'capital.loss', 'hours.per.week']
categorical_features = ['occupation', 'workclass', 'marital.status', 'relationship', 'native.country', 'education.num']
ordinal_features = ['education.num']
drop_features = ['education', 'race', 'sex']
passthrough_features = []
target = "income"

Below we have made the preprocessor we want to apply our training our model. Pipeline helps us to organize our data modifications/transformations in a nicer, elegant fashion. Each step in pipeline passes its output to the next step as input. So, for categorical_features, first the imputation is done and then we apply OHE using a pipeline.

In [11]:
preprocessor = make_column_transformer(
    (
        StandardScaler(),
        numeric_features,
    ),  # scaling on numeric features
    (
        make_pipeline(SimpleImputer(strategy="constant",
                                    fill_value="missing"),
                      OneHotEncoder(handle_unknown="ignore",
                                    sparse=False)),
        categorical_features,
    ),  # Imputation and OHE on categorical features
    (
        ("drop"),
        drop_features
    ),  # drop the drop features
)

# 7. Building ML model

In [12]:
results_dict = {}  # dictionary to store all the results

X_train = train_df_nan.drop(columns=["income"])
y_train = train_df_nan["income"]

X_test = test_df_nan.drop(columns=["income"])
y_test = test_df_nan["income"]

In [13]:
def mean_cross_val_scores(model, X_train, y_train, **kwargs):
    """
    Returns mean scores of cross validation

    Parameters
    ----------
    model :
        scikit-learn model
    X_train : numpy array or pandas DataFrame
        X in the training data
    y_train :
        y in the training data

    Returns
    ----------
        pandas Series with mean scores from cross_validation
    """

    scores = cross_validate(model, X_train, y_train, **kwargs)

    mean_scores = pd.DataFrame(scores).mean()
    out_col = []

    for i in range(len(mean_scores)):
        out_col.append(round(mean_scores[i], 3))

    return pd.Series(data=out_col, index=mean_scores.index)

## 7.1 Making a Baseline model

In [14]:
pipe = make_pipeline(preprocessor, DummyClassifier(strategy='prior')) 
results_dict = mean_cross_val_scores(pipe, X_train, y_train, cv=5, return_train_score=True)
pd.DataFrame(results_dict, columns=['DummyClassifier'])

,DummyClassifier
fit_time,0.135
score_time,0.043
test_score,0.759
train_score,0.759


The stratgey for DummmyClassifier is `prior` which predicts the class that maximizes the most frequent class and predict_proba returns the class prior.

Below we train different parametric and non-parametric models on our preprocessed data using pipelines.

# 7.2 Training multiple models

In [15]:
models = {
    "decision tree": DecisionTreeClassifier(random_state=2018),
    "kNN": KNeighborsClassifier(),
    "RBF SVM": SVC(random_state=2018),
}

models_short = {
     "decision tree": 'DecisionTree',
    "kNN": 'kNN',
    'RBF SVM': 'SVC',
}

results_dict = {}
for i in models:
    pipe_temp = make_pipeline(preprocessor, models[i])
    results_dict[models_short[i]] = mean_cross_val_scores(
    pipe_temp, X_train, y_train, cv=5, return_train_score=True)
    
pd.DataFrame(results_dict)

,DecisionTree,kNN,SVC
fit_time,0.402,0.701,32.740
score_time,0.039,12.160,5.488
test_score,0.813,0.832,0.856
train_score,1.000,0.881,0.869


- The Decision Tree overfits on the training data giving 100% accuracy on the training set. SVC has the lowest training set accuracy.  
- On the other hand, SVC gives the best validation accuracy and due to overfitting on the training data. Decision Tree has the poorest performance on the validation set.  
- All the above three models give better validation accuracies than the baseline model.   
- RBF SVC has the best validation accuracy but it is too slow! Decision Tree Classifier is the fastest one, doing most of its work during fitting the model.

# 7.3 Hyperparameter Optimization for SVC

We are optimizing hyperparameters for only SVC which gave the best validation scores in the previous section. Ideally, we would want do this on all the models and then arrive at the best model.

We are using below a kind of GridSearch to do optimization. Another way to do so is to use RandomizedSearch.

Hyperparameter used: **C**
- C is regularization parameter with squared L2 penalty. 
- The strength of the regularization is inversely proportional to C. Must be strictly positive.

In [16]:
param_grid = {"C": np.logspace(-2, 2, 4)}

results_dict = {}
for i in param_grid["C"]:
    pipe_temp = make_pipeline(preprocessor, SVC(C=i, random_state=2018))
    results_dict[i] = mean_cross_val_scores(
    pipe_temp, X_train, y_train, cv=5, return_train_score=True)
    
hyper_C = pd.DataFrame(results_dict).T
hyper_C.index.name='Hyperparameter: C'
hyper_C

,fit_time,score_time,test_score,train_score
Hyperparameter: C,,,,
0.010000,42.028,7.764,0.794,0.796
0.215443,32.148,5.785,0.855,0.859
4.641589,37.769,5.539,0.856,0.884
100.000000,137.114,5.503,0.836,0.924


In [17]:
round(hyper_C.index[hyper_C.test_score.argmax()], 3)

4.642

- C = 4.642 gives the best validation accuracy of 0.856. It is the same as the default validation accuracy which was 0.856. 
- C = 4.642 is slightly worse than the default parameter since the gap between training and validation score is lesser.
- So, we use the default parameters for the final model.

# 8. Evaluating on the test set

In [18]:
optimal_model = make_pipeline(preprocessor, SVC(random_state=2018))
optimal_model.fit(X_train, y_train)
print(f"Score on test set: { optimal_model.score(X_test, y_test):.3f}")

Score on test set: 0.857


Yay! Our final score model is close to the validation score.